In [1]:
import os
from keras.models import load_model
import numpy as np
import cv2
from keras_preprocessing.image import array_to_img, img_to_array, load_img
from keras import models
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Specify folder paths
output_base_folder = ''  # Base folder for outputs
IMAGE_SIZE = (224, 224)
target_layer = 'block5_conv3'

def grad_cam(input_model, x, layer_name):
    """
    Compute Grad-CAM for a given model, input image, and target layer.
    """
    X = np.expand_dims(x, axis=0)
    preprocessed_input = X.astype('float32') / 255.0    

    grad_model = models.Model([input_model.inputs], [input_model.get_layer(layer_name).output, input_model.output])

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(preprocessed_input)
        class_idx = np.argmax(predictions[0])
        loss = predictions[:, class_idx]

    output = conv_outputs[0]
    grads = tape.gradient(loss, conv_outputs)[0]

    gate_f = tf.cast(output > 0, 'float32')
    gate_r = tf.cast(grads > 0, 'float32')
    guided_grads = gate_f * gate_r * grads

    weights = np.mean(guided_grads, axis=(0, 1))
    cam = np.dot(output, weights)

    cam = cv2.resize(cam, IMAGE_SIZE, cv2.INTER_LINEAR)
    cam = np.maximum(cam, 0)
    heatmap = cam / cam.max()
    return heatmap

def save_grad_cam_images(model, dataset_path, output_folder, target_layer):
    """
    Apply Grad-CAM to all images in the dataset and save the resulting images.
    """
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(dataset_path):
        if filename.endswith(('.jpeg', '.png', '.jpg')):  # Supported image extensions
            image_path = os.path.join(dataset_path, filename)
            x = img_to_array(load_img(image_path, color_mode='rgb', target_size=IMAGE_SIZE))
            
            if x.ndim == 3 and x.shape[-1] == 3:
                x = np.expand_dims(x, axis=0)
            elif x.ndim == 2:
                x = np.expand_dims(x, axis=-1)
                x = np.concatenate([x] * 3, axis=-1)
                x = np.expand_dims(x, axis=0)
            else:
                raise ValueError("Unsupported image shape")
                
            heatmap = grad_cam(model, x[0], target_layer)

            jet_cam = cv2.applyColorMap(np.uint8(255.0 * heatmap), cv2.COLORMAP_JET)
            rgb_cam = cv2.cvtColor(jet_cam, cv2.COLOR_BGR2RGB)
            
            if model.input_shape[-1] == 1:
                rgb_x = np.squeeze(x, axis=0)
                rgb_x = np.concatenate([rgb_x] * 3, axis=-1)
            else:
                rgb_x = np.squeeze(x, axis=0)

            output_image = (np.float32(rgb_cam) + rgb_x / 2)

            plt.figure(figsize=(15, 10))
            plt.subplot(1, 2, 1)
            plt.title('Original Image')
            plt.imshow(array_to_img(rgb_x))
            plt.axis('off')

            plt.subplot(1, 2, 2)
            plt.title('Grad-CAM')
            plt.imshow(array_to_img(output_image))
            
            norm = mcolors.Normalize(vmin=0, vmax=1)
            sm = plt.cm.ScalarMappable(cmap="jet", norm=norm)
            sm.set_array([])
            plt.colorbar(sm, ax=plt.gca(), orientation='vertical', fraction=0.02, pad=0.04, label='Intensity')

            # Set the output folder for saving
            output_path = os.path.join(output_folder, f'grad_cam_{filename}.jpg')
            plt.savefig(output_path, format='jpg', bbox_inches='tight', transparent=False)
            plt.close()

def process_fold(fold_index):
    """
    Process a specific fold: load the model, perform Grad-CAM, and save the results.
    """
    fold_model_path = f''
    fold_test_data_path = f''
    output_folder = os.path.join(output_base_folder, f'fold_{fold_index}')
    
    # Check if paths exist
    if not os.path.exists(fold_model_path):
        print(f"Model file not found: {fold_model_path}")
        return
    if not os.path.exists(fold_test_data_path):
        print(f"Test data not found: {fold_test_data_path}")
        return
    
    # Load the model
    model = load_model(fold_model_path)

    # Save Grad-CAM results for images in the test_images folder
    save_grad_cam_images(model, fold_test_data_path, output_folder, target_layer)

# Run processing for folds 1 to 5
for i in range(1, 6):
    process_fold(i)